In [183]:
import requests
import hashlib
import datetime
import string
import csv

In [184]:
public_key = "bbca2c9c88fb26b6186f7adc5dafbc3e"
private_key = "f850ae27c606bc96740132921ab101c0e69d6511"


In [185]:
def get_TS_HASH( pub_key, priv_key):
    ts = str(datetime.datetime.now()).replace(" ", "").replace(".", "").replace("-", "").replace(":", "")
    m = hashlib.md5()
    h = ts + priv_key + pub_key
    m.update( h.encode('utf-8'))
    h = m.hexdigest()
    return ts, h
    

In [186]:
def make_url(base_url,limit,offset, pub_key, priv_key):
    ts, h = get_TS_HASH( pub_key, priv_key)
    get_url = base_url + '?ts=' + ts + '&limit=' + str(limit) + '&offset=' + str(offset) + "&apikey=" + pub_key + '&hash=' + h
    return get_url

In [187]:
filepath = 'event.csv'
out_put = open(filepath, "w+", newline='')
datafile = csv.writer(out_put) 
header = ['ID','title',  'description', 'next_event_id', 'prev_event_id']
datafile.writerow(header)

50

In [188]:
events_url = "http://gateway.marvel.com:80/v1/public/events"

In [189]:
count = 0
r = requests.get(make_url(events_url, 100, count, public_key, private_key))
if(r.status_code != 200):
    print("something wrong!!! error code of %d"% r.status_code)
data = r.json()['data']
total = data['total']

In [190]:

while(count < total):
    count = count + data['count']
    for i in range(len(data['results'])):
        row = []
        event = data['results'][i]
        row.append(event['id'])
        row.append(event['title'])
        row.append(event['description'])
        try:
            row.append(event['next']['resourceURI'].split('/')[-1])
        except:
            row.append("null")
        try:
            row.append(event['previous']['resourceURI'].split('/')[-1])
        except:
            row.append("null")
            
        datafile.writerow(row)
        
    
    r = requests.get(make_url(events_url, 100, count, public_key, private_key))
        
    if(r.status_code != 200):
        print("something wrong!!! error code of %d"% r.status_code)
        break;
    data = r.json()['data']
        
        
out_put.close()       
            
        
        
    
    